## Setup

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
print("✓ Google Drive mounted")

Mounted at /content/drive
✓ Google Drive mounted


In [ ]:
import os
os.chdir('/content')

!rm -rf automatic-pain-recognition
!git clone https://github.com/alicka33/automatic-pain-recognition.git

os.chdir('/content/automatic-pain-recognition')
print(f"✓ Repository cloned. Current directory: {os.getcwd()}")

Cloning into 'automatic-pain-recognition'...
remote: Enumerating objects: 333, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 333 (delta 28), reused 44 (delta 16), pack-reused 277 (from 1)
Receiving objects: 100% (333/333), 12.41 MiB | 16.46 MiB/s, done.
Resolving deltas: 100% (152/152), done.
✓ Repository cloned. Current directory: /content/automatic-pain-recognition


In [ ]:
!pip install -q -r requirements.txt
!pip install -q pytest pytest-cov
print("✓ Dependencies installed")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.5/253.5 kB 3.9 MB/s eta 0:00:00
✓ Dependencies installed


In [ ]:
import sys
from pathlib import Path

project_root = Path('/content/automatic-pain-recognition')
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"✓ Project path updated")

✓ Project path updated


## Run Tests

In [5]:
import os
import tempfile
import shutil
import numpy as np
import pandas as pd
from pathlib import Path

from data_preparation.process_dataset import (
    ProcessingConfig,
    ensure_dir,
    pad_or_truncate_sequence
)

In [6]:
print("\n" + "="*70)
print("TEST 1: ProcessingConfig")
print("="*70)

try:
    # Test default config
    config_default = ProcessingConfig()
    assert config_default.colab_root == '/content/drive/MyDrive/PainRecognitionProject/'
    assert config_default.max_sequence_length == 46
    assert config_default.feature_dim == 1434
    print("✓ Default ProcessingConfig created successfully")

    # Test custom config
    config_custom = ProcessingConfig(
        colab_root='/custom/root',
        max_sequence_length=50,
        feature_dim=1000
    )
    assert config_custom.colab_root == '/custom/root'
    assert config_custom.max_sequence_length == 50
    assert config_custom.feature_dim == 1000
    print("✓ Custom ProcessingConfig created successfully")

    # Test properties
    temp_root = tempfile.mkdtemp()
    config_temp = ProcessingConfig(colab_root=temp_root)
    expected_data_dir = os.path.join(temp_root, config_temp.dataset_subdir)
    assert config_temp.data_dir == expected_data_dir
    print("✓ ProcessingConfig properties work correctly")

    print("\n✓ TEST 1 PASSED")
except Exception as e:
    print(f"✗ TEST 1 FAILED: {e}")
    import traceback
    traceback.print_exc()


TEST 1: ProcessingConfig
✓ Default ProcessingConfig created successfully
✓ Custom ProcessingConfig created successfully
✓ ProcessingConfig properties work correctly

✓ TEST 1 PASSED


In [7]:
print("\n" + "="*70)
print("TEST 2: ensure_dir")
print("="*70)

try:
    # Test single directory creation
    temp_base = tempfile.mkdtemp()
    new_dir = os.path.join(temp_base, 'new_dir')

    assert not os.path.exists(new_dir)
    ensure_dir(new_dir)
    assert os.path.exists(new_dir)
    assert os.path.isdir(new_dir)
    print("✓ Single directory creation works")

    # Test nested directory creation
    nested_dir = os.path.join(temp_base, 'a', 'b', 'c', 'd')
    ensure_dir(nested_dir)
    assert os.path.exists(nested_dir)
    print("✓ Nested directory creation works")

    # Test idempotency (should not fail if dir exists)
    ensure_dir(nested_dir)
    assert os.path.exists(nested_dir)
    print("✓ ensure_dir is idempotent")

    shutil.rmtree(temp_base)
    print("\n✓ TEST 2 PASSED")
except Exception as e:
    print(f"✗ TEST 2 FAILED: {e}")
    import traceback
    traceback.print_exc()


TEST 2: ensure_dir
✓ Single directory creation works
✓ Nested directory creation works
✓ ensure_dir is idempotent

✓ TEST 2 PASSED


In [ ]:
print("\n" + "="*70)
print("TEST 3: pad_or_truncate_sequence")
print("="*70)

try:
    # Test padding
    seq_short = np.random.randn(30, 1434).astype(np.float32)
    padded = pad_or_truncate_sequence(seq_short, max_length=46, feature_dim=1434)
    assert padded.shape == (46, 1434)
    assert np.allclose(padded[:30], seq_short)
    assert np.allclose(padded[30:], 0.0)
    print("✓ Padding works correctly")

    # Test truncation
    seq_long = np.random.randn(100, 1434).astype(np.float32)
    truncated = pad_or_truncate_sequence(seq_long, max_length=46, feature_dim=1434)
    assert truncated.shape == (46, 1434)
    assert np.allclose(truncated, seq_long[:46])
    print("✓ Truncation works correctly")

    # Test exact length (no operation)
    seq_exact = np.random.randn(46, 1434).astype(np.float32)
    result = pad_or_truncate_sequence(seq_exact, max_length=46, feature_dim=1434)
    assert result.shape == (46, 1434)
    assert np.allclose(result, seq_exact)
    print("✓ Exact length sequences unchanged")

    # Test None sequence returns zero array
    result_none = pad_or_truncate_sequence(None, max_length=46, feature_dim=1434)
    assert result_none.shape == (46, 1434)
    assert np.allclose(result_none, 0)
    print("✓ None sequence returns zero-padded array")

    print("\n✓ TEST 3 PASSED")
except Exception as e:
    print(f"✗ TEST 3 FAILED: {e}")
    import traceback
    traceback.print_exc()


TEST 3: pad_or_truncate_sequence
✓ Padding works correctly
✓ Truncation works correctly
✓ Exact length sequences unchanged
✓ None sequence returns zero-padded array

✓ TEST 3 PASSED
